<a href="https://colab.research.google.com/github/haticekck/object_detection/blob/main/faster-rcnn_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!ls

drive  sample_data


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
# model
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 401MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [4]:
!unzip /content/drive/MyDrive/data.zip -d data

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: data/data/labels/train/MVI_40871_img01234.txt  
  inflating: data/__MACOSX/data/labels/train/._MVI_40871_img01234.txt  
  inflating: data/data/labels/train/MVI_40213_img00015.txt  
  inflating: data/__MACOSX/data/labels/train/._MVI_40213_img00015.txt  
  inflating: data/data/labels/train/MVI_40131_img00113.txt  
  inflating: data/__MACOSX/data/labels/train/._MVI_40131_img00113.txt  
  inflating: data/data/labels/train/MVI_40991_img00571.txt  
  inflating: data/__MACOSX/data/labels/train/._MVI_40991_img00571.txt  
  inflating: data/data/labels/train/MVI_20064_img00351.txt  
  inflating: data/__MACOSX/data/labels/train/._MVI_20064_img00351.txt  
  inflating: data/data/labels/train/MVI_40162_img01430.txt  
  inflating: data/__MACOSX/data/labels/train/._MVI_40162_img01430.txt  
  inflating: data/data/labels/train/MVI_40991_img00217.txt  
  inflating: data/__MACOSX/data/labels/train/._MVI_40991_img00217.txt  
  inflating: data/data

In [ ]:
# staring train
results = model.train(
    data='/content/data/data/data.yaml',
    epochs=50,
    imgsz=416,
    batch=16,
    save_crop=False,
    device="cuda"
)

Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=T

100%|██████████| 755k/755k [00:00<00:00, 97.8MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,628 parameters, 3,011,612 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 313MB/s]


...

      50/50      1.35G      0.697     0.3937     0.8405        106        416: 100%|██████████| 131/131 [00:24<00:00,  5.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:11<00:00,  3.96it/s]

                   all       1408      16461      0.682      0.502      0.565      0.405



50 epochs completed in 0.527 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:14<00:00,  3.13it/s]


                   all       1408      16461      0.657      0.556      0.593      0.428
                others        389        409       0.66      0.318        0.4      0.294
                   car       1395      13433      0.697      0.706      0.741      0.515
                   van        570        874      0.515      0.468      0.467       0.36
                   bus        783       1745      0.756      0.731      0.765      0.542
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train2


In [ ]:
import shutil

shutil.make_archive('train2', 'zip', '/content/runs/detect/train2')

'/content/train2.zip'

In [ ]:
from google.colab import files
files.download('/content/train2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Faster C-NN

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torchvision
import numpy as np
import os
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

In [ ]:
import os
from PIL import Image

yolo_labels_dir = '/content/data/data/labels/train'
yolo_images_dir = '/content/data/data/images/train'
rcnn_labels_dir = '/content/data/data/rcnn_labels'

os.makedirs(rcnn_labels_dir, exist_ok=True)

for label_file in os.listdir(yolo_labels_dir):
    yolo_label_path = os.path.join(yolo_labels_dir, label_file)
    image_name = label_file.replace('.txt', '.jpg')
    image_path = os.path.join(yolo_images_dir, image_name)
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue

    img = Image.open(image_path)
    W, H = img.size

    with open(yolo_label_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        cls = int(parts[0])
        x_c, y_c, w, h = map(float, parts[1:])

        xmin = int((x_c - w/2) * W)
        ymin = int((y_c - h/2) * H)
        xmax = int((x_c + w/2) * W)
        ymax = int((y_c + h/2) * H)

        # Faster R-CNN format: xmin ymin xmax ymax class
        new_line = f"{xmin} {ymin} {xmax} {ymax} {cls}\n"
        new_lines.append(new_line)

    out_path = os.path.join(rcnn_labels_dir, label_file)
    with open(out_path, 'w') as out_f:
        out_f.writelines(new_lines)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, labels_dir, transforms=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(self.images_dir)))
        self.boxes = list(sorted(os.listdir(self.labels_dir)))

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")

        box_path = os.path.join(self.labels_dir, self.boxes[idx])
        boxes = []
        labels = []
        with open(box_path) as f:
            for line in f:
                parts = line.strip().split()
                xmin, ymin, xmax, ymax, label = map(float, parts)
                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(int(label))
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            img = self.transforms(img)
        else:
            img = F.to_tensor(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import os
from PIL import Image

root_dir = '/content/data/data'
splits = ['train', 'val']

for split in splits:
    yolo_labels_dir = os.path.join(root_dir, 'labels', split)
    yolo_images_dir = os.path.join(root_dir, 'images', split)
    rcnn_labels_dir = os.path.join(root_dir, 'rcnn_labels', split)

    os.makedirs(rcnn_labels_dir, exist_ok=True)

    for label_file in os.listdir(yolo_labels_dir):
        yolo_label_path = os.path.join(yolo_labels_dir, label_file)
        image_name = label_file.replace('.txt', '.jpg')
        image_path = os.path.join(yolo_images_dir, image_name)
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        img = Image.open(image_path)
        W, H = img.size

        with open(yolo_label_path, 'r') as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            cls = int(parts[0])
            x_c, y_c, w, h = map(float, parts[1:])

            xmin = int((x_c - w/2) * W)
            ymin = int((y_c - h/2) * H)
            xmax = int((x_c + w/2) * W)
            ymax = int((y_c + h/2) * H)

            # Faster R-CNN format: xmin ymin xmax ymax class
            new_line = f"{xmin} {ymin} {xmax} {ymax} {cls}\n"
            new_lines.append(new_line)

        out_path = os.path.join(rcnn_labels_dir, label_file)
        with open(out_path, 'w') as out_f:
            out_f.writelines(new_lines)

In [ ]:
train_images_dir = '/content/data/data/images/train'
train_labels_dir = '/content/data/data/rcnn_labels/train'
val_images_dir = '/content/data/data/images/val'
val_labels_dir = '/content/data/data/rcnn_labels/val'

train_dataset = CustomDataset(train_images_dir, train_labels_dir)
val_dataset = CustomDataset(val_images_dir, val_labels_dir)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x))
)

val_loader = DataLoader(
    val_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x))
)

In [ ]:
num_classes = 5

# Pretrained model
model = fasterrcnn_resnet50_fpn(weights="DEFAULT")

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 179MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

In [ ]:
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
from tqdm import tqdm

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    print(f"Epoch {epoch+1}/{num_epochs}")
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        epoch_loss += losses.item()

    if 'lr_scheduler' in locals():
        lr_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs} - Avg Loss: {epoch_loss/len(train_loader):.4f}")

Epoch 1/50


Epoch 1: 100%|██████████| 1030/1030 [10:01<00:00,  1.71it/s]


Epoch 1/50 - Avg Loss: 0.3905
Epoch 2/50


Epoch 2:   2%|▏         | 19/1030 [00:11<10:21,  1.63it/s]


KeyboardInterrupt: 

In [ ]:
from torch.utils.data import DataLoader, Subset
import numpy as np
from tqdm import tqdm

num_epochs = 10
n_samples = 250

for epoch in range(num_epochs):
    indices = np.random.choice(len(train_dataset), n_samples, replace=False)
    subset_train_dataset = Subset(train_dataset, indices)
    train_loader = DataLoader(
        subset_train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x))
    )

    model.train()
    epoch_loss = 0
    print(f"Epoch {epoch+1}/{num_epochs}")

    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        epoch_loss += losses.item()

    if 'lr_scheduler' in locals():
        lr_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs} - Avg Loss: {epoch_loss/len(train_loader):.4f}")

In [ ]:
images_dir = '/content/data/data/images/train'
labels_dir = '/content/data/data/rcnn_labels/train'  # Dönüştürülmüş label yolunu kullan!

image_files = set([os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')])
label_files = set([os.path.splitext(f)[0] for f in os.listdir(labels_dir)])

images_without_labels = image_files - label_files
print("Label'ı olmayan image dosyaları:", images_without_labels)

Label'ı olmayan image dosyaları: {'MVI_39811_img00855', 'MVI_39811_img00376', 'MVI_39931_img00709', 'MVI_39931_img00750', 'MVI_63544_img01000', 'MVI_39931_img00816', 'MVI_39811_img00991', 'MVI_39811_img00903', 'MVI_39811_img01036', 'MVI_40213_img01287', 'MVI_39811_img00392', 'MVI_40991_img00277', 'MVI_39811_img00365', 'MVI_39761_img01024', 'MVI_40991_img01614', 'MVI_39811_img00241', 'MVI_39931_img00992', 'MVI_39811_img00352', 'MVI_40162_img00344', 'MVI_39851_img00294', 'MVI_39851_img00167', 'MVI_39851_img00232', 'MVI_40991_img01605', 'MVI_40162_img00348', 'MVI_40992_img01579', 'MVI_63544_img00998', 'MVI_63544_img00917', 'MVI_63544_img01034', 'MVI_39931_img00726', 'MVI_39761_img00920', 'MVI_39761_img01371', 'MVI_39761_img01001', 'MVI_39931_img00730', 'MVI_40991_img01644'}


In [ ]:
for img_base in images_without_labels:
    for ext in ['.jpg', '.png']:
        img_path = os.path.join(images_dir, img_base + ext)
        if os.path.exists(img_path):
            os.remove(img_path)
            print(f"Silindi: {img_path}")

Silindi: /content/data/data/images/train/MVI_39811_img00855.jpg
Silindi: /content/data/data/images/train/MVI_39811_img00376.jpg
Silindi: /content/data/data/images/train/MVI_39931_img00709.jpg
Silindi: /content/data/data/images/train/MVI_39931_img00750.jpg
Silindi: /content/data/data/images/train/MVI_63544_img01000.jpg
Silindi: /content/data/data/images/train/MVI_39931_img00816.jpg
Silindi: /content/data/data/images/train/MVI_39811_img00991.jpg
Silindi: /content/data/data/images/train/MVI_39811_img00903.jpg
Silindi: /content/data/data/images/train/MVI_39811_img01036.jpg
Silindi: /content/data/data/images/train/MVI_40213_img01287.jpg
Silindi: /content/data/data/images/train/MVI_39811_img00392.jpg
Silindi: /content/data/data/images/train/MVI_40991_img00277.jpg
Silindi: /content/data/data/images/train/MVI_39811_img00365.jpg
Silindi: /content/data/data/images/train/MVI_39761_img01024.jpg
Silindi: /content/data/data/images/train/MVI_40991_img01614.jpg
Silindi: /content/data/data/images/train

In [3]:
!pip install pycocotools

In [7]:
!unzip /content/drive/MyDrive/data2.zip -d data2

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: data2/data/rcnn_labels/train/MVI_40871_img01234.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40871_img01234.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40213_img00015.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40213_img00015.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40131_img00113.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40131_img00113.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40991_img00571.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40991_img00571.txt  
  inflating: data2/data/rcnn_labels/train/MVI_20064_img00351.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_20064_img00351.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40162_img01430.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40162_img01430.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40991_img00217.txt  
  inflating: dat

In [9]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
from tqdm import tqdm

# Cihaz
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Modeli başlat (COCO değil)
model = fasterrcnn_resnet50_fpn(pretrained=False)

# Sınıf sayını belirle
num_classes = 5  # 4 sınıf + 1 background

# Kendi sınıf sayına göre box predictor’ı güncelle
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Eğitilmiş ağırlıkları yükle
model.load_state_dict(torch.load('/content/drive/MyDrive/fasterrcnn_final.pth', map_location=device))
model.to(device)
model.eval()

# === Klasörler ===
image_folder = '/content/data2/data/images/val'
label_folder = '/content/data2/data/rcnn_labels/val'  # .txt dosyalarının olduğu klasör

# === IoU hesaplayıcı ===
def compute_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter + 1e-6
    return inter / union

# === Her görselin label dosyasını oku ===
def load_labels(label_path):
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = list(map(float, line.strip().split()))
            x1, y1, x2, y2 = parts[:4]
            bboxes.append([x1, y1, x2, y2])
    return bboxes

# === Transform ===
transform = T.Compose([T.ToTensor()])

TP, FP, FN = 0, 0, 0
iou_threshold = 0.5

for img_name in tqdm(os.listdir(image_folder)):
    if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # 1. Görseli yükle
    img_path = os.path.join(image_folder, img_name)
    image = Image.open(img_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)

    # 2. Ground truth bboxları oku
    txt_name = os.path.splitext(img_name)[0] + '.txt'
    label_path = os.path.join(label_folder, txt_name)
    if not os.path.exists(label_path):
        continue  # Etiket yoksa atla
    gt_boxes = load_labels(label_path)
    matched = np.zeros(len(gt_boxes))

    # 3. Tahmin al
    with torch.no_grad():
        outputs = model(image_tensor)
    pred_boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    pred_boxes = pred_boxes[scores > 0.5]  # eşik uygula

    # 4. IoU eşleştirme
    for pred_box in pred_boxes:
        ious = [compute_iou(pred_box, gt_box) for gt_box in gt_boxes]
        if not ious:
            FP += 1
            continue

        max_iou = max(ious)
        max_idx = np.argmax(ious)
        if max_iou >= iou_threshold and matched[max_idx] == 0:
            TP += 1
            matched[max_idx] = 1
        else:
            FP += 1

    FN += len(gt_boxes) - sum(matched)

# === Sonuçlar ===
precision = TP / (TP + FP + 1e-6)
recall = TP / (TP + FN + 1e-6)
f1 = 2 * precision * recall / (precision + recall + 1e-6)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

100%|██████████| 1408/1408 [03:07<00:00,  7.53it/s]

Precision: 0.7101
Recall:    0.8388
F1 Score:  0.7691
